####  This code produces a `.csv` file that will contain all the clusters, the bands in each cluster, and the corresponding images and noise files along with the zeropoints. 
####   First, lets get the files

In [1]:
import os
import sys
import platform
import pandas as pd 
import math
import glob
import csv

from astropy.io import ascii,fits
import matplotlib.pyplot as plt

In [2]:
os_check = platform.platform(terse=True)[:5]
if os_check == 'macOS':
    preamble = '/path/to/mac/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'
else:
    preamble = '/path/to/linux/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'

In [3]:
clusters = sorted(['SpARCS0035', 'SpARCS0219','SpARCS0335','SpARCS1034','SpARCS1051','SpARCS1616','SpARCS1634','SpARCS1638','SPT0546', 'SPT0205', 'SPT2106']) # <-- the clusters 
clusters

['SPT0205',
 'SPT0546',
 'SPT2106',
 'SpARCS0035',
 'SpARCS0219',
 'SpARCS0335',
 'SpARCS1034',
 'SpARCS1051',
 'SpARCS1616',
 'SpARCS1634',
 'SpARCS1638']

In [4]:
def print_band(cluster, data_file_list, weight_file_list, prefix, suffix):
  
    for i in range(len(data_file_list)):
        file_1 = data_file_list[i]
        file_2 = weight_file_list[i]
        result = trim_mos_fits(file_1, prefix, suffix)
        
        #print(cluster, result)  # Output: example_filename
        clust_list.append(cluster)
        bands_list.append(result)
        data_files_list.append(file_1)
        noise_files_list.append(file_2)

def print_band_2(cluster, data_file_list, prefix, suffix):
  
    for i in range(len(data_file_list)):
        file_1 = data_file_list[i]
        result = trim_mos_fits(file_1, prefix, suffix)
        
        #print(cluster, result)  # Output: example_filename
        clust_list.append(cluster)
        bands_list.append(result)
        psf_match_data_files_list.append(file_1)
        


def trim_mos_fits(input_string, prefix, suffix):
    if input_string.startswith(prefix) and input_string.endswith(suffix):
        # Remove "mos" from the beginning and ".fits" from the end
        int_string = input_string.replace(prefix, '')
        final_string = int_string.replace(suffix, '')
        return final_string
        #return input_string[3:-5]
    #else:
        #raise ValueError("The input string does not start with 'mos' and end with '.fits'.")


def file_table():

    for cluster in clusters:
        data_path = f'{root2}{cluster}/GOGREEN_IMAGES/native/images/'
        prefix = f'{data_path}mos_'
        suffix = '_3.fits'
        files = sorted(glob.glob(os.path.join(f'{data_path}', '*.fits')))
        print_band(cluster, files, prefix, suffix)
        #cluster_dic[cluster] = cluster_files

    
    df['Cluster'] = clust_list
    df['Band'] = bands_list
    df['File'] = files_list

# this function is currently in cat_match_V2
def read_file_columns(file_path):
	columns = []
	
	with open(file_path, 'r') as file:
		for line in file:
			# Skip lines starting with '#'
			if line.startswith("#"):
				continue
			
			# Split the line into values and strip any extra spaces/newlines
			values = line.strip().split()
			
			# Initialize columns on the first valid line
			if not columns:
				columns = [[] for _ in values]
			
			# Append each value to its corresponding column
			for i, value in enumerate(values):
				columns[i].append(value)
	
	return columns
def modify_string(s):
	if s.startswith("SPTCL-"):
		return s.replace("SPTCL-", "SPT", 1)
	elif s.startswith("SpARCS-"):
		return s.replace("SpARCS-", "SpARCS", 1)
	return s  # Return the original string if no match

In [5]:
def make_dir(directory_path, folder_name, sub_folders):
    main_folder_path = os.path.join(directory_path, folder_name)
    
    # Check if the main folder already exists
    if not os.path.exists(main_folder_path):
        os.makedirs(main_folder_path)
    else:
        print(f"Folder already exists: {main_folder_path}")
    
    # Iterate through subfolders
    for subfolder in sub_folders:
        subfolder_path = os.path.join(main_folder_path, subfolder)
        data_path = os.path.join(subfolder_path, 'data_cutouts')
        noise_path = os.path.join(subfolder_path, 'noise_cutouts')
        decon_images_path = os.path.join(subfolder_path, 'deconvoled_images')
        decon_fits_path = os.path.join(subfolder_path, 'deconvolved_fits')
        
        # Create subfolders only if they don't exist
        if not os.path.exists(data_path):
            os.makedirs(data_path)
        else:
            print(f"Subfolder already exists: {data_path}")

        if not os.path.exists(noise_path):
            os.makedirs(noise_path)
        else:
            print(f"Subfolder already exists: {noise_path}")

        if not os.path.exists(decon_images_path):
            os.makedirs(decon_images_path)
        else:
            print(f"Subfolder already exists: {decon_images_path}")
        
        if not os.path.exists(decon_fits_path):
            os.makedirs(decon_fits_path)
        else:
            print(f"Subfolder already exists: {decon_fits_path}")

In [ ]:
#clusters2 = cluster_images['Cluster'].unique()

#### There are some clusters we dont have data for in the GoGreen cut we are using. Let us remove them

In [6]:
remove_clust = ['SpARCS0034', 'SpARCS0036','SpARCS0215', 'SpARCS1047', 'SpARCS1613']
clusters = sorted([x for x in clusters if x not in remove_clust])
plot_dir = f'{root2}Plots'
star_cut_cats_dir = f'{root2}star_cut_catalogs'
display(clusters)

['SPT0205',
 'SPT0546',
 'SPT2106',
 'SpARCS0035',
 'SpARCS0219',
 'SpARCS0335',
 'SpARCS1034',
 'SpARCS1051',
 'SpARCS1616',
 'SpARCS1634',
 'SpARCS1638']

In [7]:
cluster_df = pd.DataFrame()
clust_list = []
bands_list = []
data_files_list = []
noise_files_list = []
df = pd.DataFrame()
for cluster in clusters:
    data_path = f'{root2}{cluster}/GOGREEN_IMAGES/native/images/'
    noise_path = f'{root2}{cluster}/GOGREEN_IMAGES/native/weights/'
    prefix = f'{data_path}mos_'
    suffix = '_3.fits'
    #noise_suffix - '_3.weight.fits'
    cluster_files = sorted(glob.glob(os.path.join(f'{data_path}', '*.fits')))
    weight_files = sorted(glob.glob(os.path.join(f'{noise_path}', '*.fits')))

    #print(cluster_files)
    print_band(cluster, cluster_files, weight_files ,prefix, suffix)
    #cluster_dic[cluster] = cluster_files

In [8]:
cluster_df['Cluster'] = clust_list
cluster_df['Band'] = bands_list
cluster_df['Data File'] = data_files_list
cluster_df['Noise File'] = noise_files_list
#cluster_df.to_csv('cluster_images.csv')
#display(cluster_df['Cluster'].unique())
#cluster_df[cluster_df['Cluster'] == 'SPT0205']

In [21]:
cluster_df = pd.DataFrame()
clust_list = []
bands_list = []
psf_match_data_files_list = []
psf_match_noise_files_list = []
df = pd.DataFrame()
for cluster in clusters:
    data_path = f'{root2}{cluster}/GOGREEN_IMAGES/psf_matched/images/'
    #noise_path = f'{root2}{cluster}/GOGREEN_IMAGES/psf_matched/weights/'
    prefix = f'{data_path}mos_'
    suffix = '.psf_3.fits'
    #noise_suffix - '_3.weight.fits'
    cluster_files = sorted(glob.glob(os.path.join(f'{data_path}', '*.fits')))
    #weight_files = sorted(glob.glob(os.path.join(f'{noise_path}', '*.fits')))

    #print(cluster_files)
    print_band_2(cluster, cluster_files, prefix, suffix)
    #cluster_dic[cluster] = cluster_files
#cluster_df['Cluster'] = clust_list
#cluster_df['Band'] = bands_list
#cluster_df['PSFM Data File'] = psf_match_data_files_list
#cluster_df['PSFM Noise File'] = psf_match_noise_files_list

#display(cluster_df) 

In [11]:
len(psf_match_data_files_list)

141

In [23]:
final_files_bands = f'{root2}final_file_merge_w_psf.csv'
cluster_df = pd.read_csv(final_files_bands)
#cluster_df['PSFM Data File'] = 

In [25]:
psf_matched_dic = {'Cluster': clust_list, 'Band':bands_list, 'PSFM Data File':psf_match_data_files_list }
psf_matched_tables = pd.DataFrame(psf_matched_dic)
psf_matched_tables[psf_matched_tables['Band'] != None]
#psf_matched_tables[(psf_matched_tables['Cluster'] == 'SpARCS1034')]

,Cluster,Band,PSFM Data File
0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
2,SPT0205,None,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
3,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
4,SPT0205,None,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
...,...,...,...
136,SpARCS1638,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
137,SpARCS1638,Suprimey,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
138,SpARCS1638,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
139,SpARCS1638,None,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [30]:
psf_matched_tables[psf_matched_tables['Cluster'] == 'SpARCS1034']['PSFM Data File'][78]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/SpARCS1034/GOGREEN_IMAGES/psf_matched/images/mos_HSCy.psf_3.fits'

In [31]:
psfm_final_merge = pd.merge(cluster_df, psf_matched_tables,on=['Cluster', 'Band'])
psfm_final_merge

,Unnamed: 0,Cluster,Band,Data File,Noise File,zero_point,cPHOTID,PSF File,PSFM Data File
0,0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.92,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
1,1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.82,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
2,2,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.17,['101001571'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
3,5,SPT0205,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52,"['101001930','101001473']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
4,6,SPT0205,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.07,"['101001662','101002234']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
...,...,...,...,...,...,...,...,...,...
87,125,SpARCS1638,Suprimei,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.05,['112008857'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
88,126,SpARCS1638,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
89,127,SpARCS1638,Suprimey,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.20,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
90,128,SpARCS1638,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [32]:
psfm_final_merge[psfm_final_merge['Cluster'] == 'SpARCS1034']

,Unnamed: 0,Cluster,Band,Data File,Noise File,zero_point,cPHOTID,PSF File,PSFM Data File
54,72,SpARCS1034,HSCy,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,26.75,"['108003171','108004662']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
55,73,SpARCS1034,HSCz,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,26.70,['108004662'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
56,78,SpARCS1034,Suprimeg,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
57,79,SpARCS1034,Suprimei,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.05,['108004065'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
58,80,SpARCS1034,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
59,81,SpARCS1034,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.11,['108004662'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
60,82,SpARCS1034,WIRCamK,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,28.38,"['108001350','108004064','108004662','108008541']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [ ]:
psfm_final_merge.to_csv('psfm_final_merge.csv')

In [ ]:
psfm_final_merge['PSFM Data File'][89]
psfm_final_merge[psfm_final_merge['Cluster'] == 'SPT0205']['P']

,Unnamed: 0,Cluster,Band,Data File,Noise File,zero_point,cPHOTID,PSF File,PSFM Data File
0,0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.92,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
1,1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.82,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
2,2,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.17,['101001571'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
3,5,SPT0205,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52,"['101001930','101001473']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
4,6,SPT0205,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.07,"['101001662','101002234']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
5,7,SPT0205,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.72,"['101001571','101000435']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
6,8,SPT0205,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.27,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
7,9,SPT0205,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.13,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
8,10,SPT0205,VIMOSz,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.20,"['101001571','101001950','101001998']",/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [57]:
psfm_final_merge[psfm_final_merge['Cluster'] == 'SPT0205']['PSFM Data File'][8]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/SPT0205/GOGREEN_IMAGES/psf_matched/images/mos_VIMOSz.psf_3.fits'

In [ ]:
cluster_df[cluster_df['Cluster'] == 'SPT0546']

,Cluster,Band,Data File,Noise File
97,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
98,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
99,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
100,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
101,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
102,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
103,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
104,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
105,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
106,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [9]:
test = cluster_df[cluster_df['Cluster'] == 'SPT0546']
test

,Cluster,Band,Data File,Noise File
97,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
98,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
99,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
100,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
101,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
102,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
103,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
104,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
105,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
106,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [11]:
test['Noise File'][97]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/SPT0546/GOGREEN_IMAGES/native/weights/mos_FOURSTARJ1_3.weight.fits'

Next, let us pull in the `MAG_ZPs_cal.list` file from the GoGreen data

In [12]:
# obtains data from MAGZPs_cal.list
# Example usage:
file_path = f"{root2}MAGZPs_cal.list"
columns = read_file_columns(file_path)
#print(columns)

clusters2 = columns[0]
clusters_fixed = [modify_string(cluster) for cluster in clusters2]

bands = columns[1]
zero_points = columns[2]
MAGZ_dict = {'Cluster':clusters_fixed,'Band': bands, 'zero_point': zero_points}
magz_df = pd.DataFrame(MAGZ_dict)
display(magz_df)

,Cluster,Band,zero_point
0,SPT0205,VIMOSU,32.27
1,SPT0205,VIMOSB,33.52
2,SPT0205,VIMOSV,33.13
3,SPT0205,VIMOSR,33.72
4,SPT0205,VIMOSI,33.07
...,...,...,...
141,SpARCS1613,K,26.56
142,SpARCS1613,IRAC1,21.55
143,SpARCS1613,IRAC2,21.55
144,SpARCS1613,IRAC3,21.55


In [13]:
magz_df[magz_df['Cluster'] == 'SPT0546']

,Cluster,Band,zero_point
9,SPT0546,VIMOSU,32.00
10,SPT0546,VIMOSB,33.52
11,SPT0546,VIMOSV,33.21
12,SPT0546,VIMOSR,33.78
13,SPT0546,VIMOSI,33.14
14,SPT0546,VIMOSz,32.17
15,SPT0546,FOURSTARJ1,30.75
16,SPT0546,FOURSTARJ,30.80
17,SPT0546,FOURSTARKs,29.31


#### now that we have the MAGZP's and the file list imported, let's merge them

In [14]:
magz_files_merged = pd.merge(cluster_df,magz_df, how='outer', on=['Cluster', 'Band'] )
magz_files_merged[magz_files_merged['Cluster']== 'SPT0546']

,Cluster,Band,Data File,Noise File,zero_point
11,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.80
12,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.75
13,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.31
14,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
15,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
16,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52
17,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.14
18,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.78
19,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.00
20,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.21


#### Let's clean this up a bit, there are some values that do not have files associated with them for the particular bands. Let's find them. First let's drop all the clusters in `remove_clust`

In [15]:
magz_files_merged_filtered =  magz_files_merged[~magz_files_merged['Cluster'].isin(remove_clust)]
magz_files_merged_filtered

,Cluster,Band,Data File,Noise File,zero_point
0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.92
1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.82
2,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.17
3,SPT0205,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
4,SPT0205,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
...,...,...,...,...,...
180,SpARCS1638,Suprimei,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.05
181,SpARCS1638,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00
182,SpARCS1638,Suprimey,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.20
183,SpARCS1638,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.00


#### Next, let us check to see if the remaining have files. Notice a few don't have zeropoint, but that's OK since we are using a zeropoint = 25 for all the GoGreen clusters

In [16]:
magz_files_merged_filtered[magz_files_merged_filtered['Data File'].isna()]

,Cluster,Band,Data File,Noise File,zero_point


#### Notice that these 2 clusters do not have files in those bands. In fact, SpARCS0335 Ks band comes from FOURSTAR, and SpARCS1616 z-band comes from HyperSuprimeCam, so we will drop these rows as well. 
#### We could have dropped both in one go, but I found it useful to see this for exploratory purposes

In [17]:
magz_files_merged_missing_filtered = magz_files_merged_filtered[~magz_files_merged_filtered['Data File'].isna()]
magz_files_merged_missing_filtered

,Cluster,Band,Data File,Noise File,zero_point
0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.92
1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.82
2,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.17
3,SPT0205,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
4,SPT0205,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
...,...,...,...,...,...
180,SpARCS1638,Suprimei,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.05
181,SpARCS1638,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00
182,SpARCS1638,Suprimey,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.20
183,SpARCS1638,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.00


In [21]:
magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == 'SPT0546']

,Cluster,Band,Data File,Noise File,zero_point
11,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.80
12,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.75
13,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.31
14,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
15,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
16,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52
17,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.14
18,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.78
19,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.00
20,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.21


In [22]:
magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == 'SPT0546']['Noise File'][11]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/SPT0546/GOGREEN_IMAGES/native/weights/mos_FOURSTARJ_3.weight.fits'

#### HORRAY! This dataframe will be the file we will import most of the time to do things like run `sewpy` to get FWHM, run EPSFBuilder to build the PSF, and to make cutouts of galaxies
#### to run SUNet deconvolution. Save this DataFrame to a `.csv` and keep it handy. 

#### ONE LAST THING: it is also good to have the list of "bad stars" from visual inspection in each band as well. This is not a step we want to do more than once. I have a list of stars I thought
#### were bad, and would like to merge that list as well

In [ ]:
# Add the code that created `psfreject.csv`

In [23]:
reject_ids = pd.read_csv(f'{root2}psfreject.csv', sep="|")
reject_ids['Cluster'].unique()

array(['SpARCS0035', 'SpARCS0219', 'SpARCS0335', 'SpARCS1034',
       'SpARCS1051', 'SpARCS1616', 'SpARCS1634', 'SpARCS1638', 'SPT0205',
       'SPT0546', 'SPT2106'], dtype=object)

In [24]:
reject_ids[reject_ids['Cluster'] == 'SPT0546']

,Cluster,Band,cPHOTID
108,SPT0546,FOURSTARJ,['None']
109,SPT0546,FOURSTARJ1,['None']
110,SPT0546,FOURSTARKs,['None']
111,SPT0546,IRAC1,['None']
112,SPT0546,IRAC2,['None']
113,SPT0546,VIMOSB,['None']
114,SPT0546,VIMOSI,"['102000633','102000682','102000732','10200085..."
115,SPT0546,VIMOSR,"['102000633','102000682','102001846','10200196..."
116,SPT0546,VIMOSU,['102003074']
117,SPT0546,VIMOSV,"['102001167','102002393','102002474','10200273..."


In [20]:
magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == 'SPT0546']

,Cluster,Band,Data File,Noise File,zero_point
11,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.80
12,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.75
13,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.31
14,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
15,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN
16,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52
17,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.14
18,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.78
19,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.00
20,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.21


In [25]:
final_merge = pd.merge(magz_files_merged_missing_filtered, reject_ids, on = ['Cluster', 'Band'])
final_merge[final_merge['Cluster'] == 'SPT0546']

,Cluster,Band,Data File,Noise File,zero_point,cPHOTID
11,SPT0546,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.80,['None']
12,SPT0546,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.75,['None']
13,SPT0546,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.31,['None']
14,SPT0546,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None']
15,SPT0546,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None']
16,SPT0546,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.52,['None']
17,SPT0546,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.14,"['102000633','102000682','102000732','10200085..."
18,SPT0546,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.78,"['102000633','102000682','102001846','10200196..."
19,SPT0546,VIMOSU,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,32.00,['102003074']
20,SPT0546,VIMOSV,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.21,"['102001167','102002393','102002474','10200273..."


In [26]:
final_merge[final_merge['Cluster'] == 'SPT0546']['Noise File'][11]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/SPT0546/GOGREEN_IMAGES/native/weights/mos_FOURSTARJ_3.weight.fits'

In [28]:
reject_ids[reject_ids['Cluster'] == 'SPT0546']

,Cluster,Band,cPHOTID
108,SPT0546,FOURSTARJ,['None']
109,SPT0546,FOURSTARJ1,['None']
110,SPT0546,FOURSTARKs,['None']
111,SPT0546,IRAC1,['None']
112,SPT0546,IRAC2,['None']
113,SPT0546,VIMOSB,['None']
114,SPT0546,VIMOSI,"['102000633','102000682','102000732','10200085..."
115,SPT0546,VIMOSR,"['102000633','102000682','102001846','10200196..."
116,SPT0546,VIMOSU,['102003074']
117,SPT0546,VIMOSV,"['102001167','102002393','102002474','10200273..."


## Troubleshooting

In [ ]:
#### This function creates the directory structure I used to store the cutouts. Inside a main folder, you will have each cluster. then you have each band, where we will store the data, noise, the deconvolved images and deconvolved .fits files 

## Use this snippet to change the name of files on macOS...add it to functions that take in the filenames (making folders, cutouts, psf generation, deconvolution)

In [65]:
os_check_test = 'macOS'  # <--- used to test

In [ ]:
for cluster in clusters:
    filtered_df = magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == cluster]
    for i in filtered_df.index:
        prefix = '/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/'
        new_prefix = '/Users/lordereinion/OneDrive - University of Kansas/Deconvolution/'
        filename = filtered_df['Data File'][i]
        if os_check_test == 'macOS':
            if filename.startswith(prefix):
                filename = filename.replace(prefix, new_prefix, 1)
                print(cluster, filename)
        else:
            print(cluster, filename)
        

In [ ]:
magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == cluster]

In [57]:
mac_root = '/Users/lordereinion/OneDrive - University of Kansas/Deconvolution/'

In [42]:
psf_list = []
for cluster in sorted(clusters):
    filtered_df = cluster_df[cluster_df['Cluster'] == cluster]
    for i in filtered_df.index:
        band = filtered_df['Band'][i]
        psf_file_name = f'{root2}PSFs/{cluster}_{band}_psf.fits'
        psf_list.append(psf_file_name)
final_merge['PSF File'] = psf_list
   

In [43]:
final_merge

,Cluster,Band,Data File,Noise File,zero_point,cPHOTID,PSF File
0,SPT0205,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.92,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
1,SPT0205,FOURSTARJ1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.82,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
2,SPT0205,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.17,['101001571'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
3,SPT0205,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
4,SPT0205,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
...,...,...,...,...,...,...,...
125,SpARCS1638,Suprimei,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.05,['112008857'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
126,SpARCS1638,Suprimer,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
127,SpARCS1638,Suprimey,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.20,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
128,SpARCS1638,WIRCamJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,29.00,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [45]:
final_merge['PSF File'][125]

'/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/PSFs/SpARCS1638_Suprimei_psf.fits'

In [ ]:
for cluster in clusters:
    filtered_df = magz_files_merged_missing_filtered[magz_files_merged_missing_filtered['Cluster'] == cluster]
    for i in filtered_df.index:
        bands = filtered_df['Band']
        parent_directory = f'{root2}'
        make_dir(parent_directory, cluster, bands)
        #print(band)

In [50]:
final_merge.to_csv('final_file_merge_w_psf.csv')

In [33]:
test = final_merge[final_merge['Cluster'] == 'SpARCS0335']
test

,Cluster,Band,Data File,Noise File,zero_point,cPHOTID,PSF File
59,SpARCS0335,FOURSTARJ,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.93,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
60,SpARCS0335,FOURSTARKs,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,30.58,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
61,SpARCS0335,HAWKIY,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,31.66,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
62,SpARCS0335,IRAC1,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
63,SpARCS0335,IRAC2,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
64,SpARCS0335,IRAC3,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
65,SpARCS0335,IRAC4,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,NaN,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
66,SpARCS0335,VIMOSB,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.56,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
67,SpARCS0335,VIMOSI,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.23,['106000822'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...
68,SpARCS0335,VIMOSR,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...,33.95,['None'],/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deco...


In [36]:
len(psf_list)

130